# **차량 공유업체의 차량 파손 여부 분류하기**

## 0.미션

* 1) 미션1 : Data Preprocessing
    - **과제 수행 목표**
        - 본인의 구글 드라이브에 모델링 수행을 위해 적절한 폴더 및 파일로 **일관성 있게 정리**해야 합니다.
        - 제공된 데이터 : Car_Images.zip
            * Car_Images : 차량의 정상/파손 이미지 무작위 수집

* 2) 미션2 : CNN 모델링
    - **과제 수행 목표**
        - Tensorflow Keras를 이용하여 모델을 3개 이상 생성하세요.
            - 모델 구조와 파라미터는 자유롭게 구성하세요.
            - 단, 세부 목차에서 명시한 부분은 지켜주세요.

* 3) 미션3 : Data Argumentation & Transfer Learning
    - **과제 수행 목표**
        - 성능 개선을 위해 다음의 두가지를 시도하세요.
            * Data Augmentation을 적용하세요.(Image Generator)
            * Transfer Learning(VGG16)


## 1.환경설정 

### (1) 데이터셋 폴더 생성
- **세부요구사항**
    - C드라이브에 Datasets라는 폴더를 만드세요.
        - 구글드라이브를 사용하는경우 드라이브 첫 화면에 Datasets 라는 폴더를 만드세요. ('/content/drive/MyDrive/Datasets/')
    - 해당 폴더 안에 Car_Images.zip 파일을 넣으세요.

* 구글 Colab을 이용하는 경우

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### (2) 데이터셋 불러오기 
- **세부요구사항**
    - Car_Images.zip 파일을 C:/Datasets/ 경로에 압축 해제합니다.
    - zipfile 모듈을 이용하거나 다른 방식을 사용해도 됩니다.
        - 참고 자료 : [zipfile document](https://docs.python.org/3/library/zipfile.html#zipfile-objects)
    - 폴더구조(로컬)
        * C:/Datasets/ : 압축파일
        * C:/Datasets/Car_Images_train/ : 압축 해제한 이미지 저장소
    - 폴더구조(구글드라이브브)
        * /content/drive/MyDrive/Datasets/ : 압축파일
        * /content/drive/MyDrive/Datasets/Car_Images_train/ : 압축 해제한 이미지 저장소
    - 압축을 해제하면 다음과 같은 두 하위 폴더가 생성됩니다.
        * normal, abnormal : 각 폴더에는 이미지들이 있습니다.
        * 이후 단계에서 해당 경로로 부터 validation, test 셋을 추출하게 됩니다.
        

In [2]:
import zipfile
import os
import glob
import numpy as np

In [3]:
# 압축파일 경로
# 구글 드라이브인 경우 경로에 맞게 지정하세요.
# dataset_path  = '/content/drive/MyDrive/Datasets/'
dataset_path = 'C:/Datasets/'

file_path = dataset_path + 'Car_Images.zip'

In [4]:
# 압축 해제
# data = zipfile.ZipFile(file_path)
# data.extractall(/content/drive/MyDrive/Datasets/)

### (3) 이미지 저장을 위한 폴더 생성
- **세부요구사항**
    - train, validation, test 을 위해 각각 하위 폴더 normal과 abnormal를 준비합니다.
        - train
            * 정상 이미지 저장소 : C:/Datasets/Car_Images_train/normal/ 
                * 구글드라이브 :   /content/drive/MyDrive/Datasets/Car_Images_train/normal/
            * 파손 이미지 저장소 : C:/Datasets/Car_Images_train/abnormal/
                * 구글드라이브 : /content/drive/MyDrive/Datasets/Car_Images_train/abnormal/
        - val, test 역시 동일한 구조로 생성합니다.
    - 직접 탐색기에서 폴더를 생성할 수도 있고, os 모듈을 이용하여 코드로 작성할 수도 있습니다.
        - 참고 자료 : [os document](https://docs.python.org/3/library/os.html)

In [ ]:

# 각각 경로 지정



# train 폴더는 압축을 해제하면서 이미 생성 되어 있습니다.
os.makedirs('/content/drive/MyDrive/datasets/Car_Images_train/normal')
os.makedirs('/content/drive/MyDrive/datasets/Car_Images_train/abnormal')

# test 폴더 만들기 os.mkdir()
os.makedirs('/content/drive/MyDrive/datasets/Car_Images_test/normal')
os.makedirs('/content/drive/MyDrive/datasets/Car_Images_test/abnormal')

# validation 폴더 만들기
os.makedirs('/content/drive/MyDrive/datasets/Car_Images_val/normal')
os.makedirs('/content/drive/MyDrive/datasets/Car_Images_val/abnormal')

## 2.데이터 전처리

### (1) 데이터 분할 : Training set | Validation set | Test set 생성
- **세부요구사항**
    - Training set, Validation set, Test set을 만듭니다.
        * size
            * test : 전체에서 20%를 추출합니다.
            * validation : test를 떼어낸 나머지에서 다시 20%를 추출합니다.
        * 데이터는 랜덤하게 추출해야 합니다.
            - random, shutil 모듈을 이용하여 랜덤하게 추출할 수 있습니다.
                - [random document](https://docs.python.org/3/library/random.html) | [shutil document](https://docs.python.org/3/library/shutil.html)
            * 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

#### 1) test, validation 크기를 지정

In [ ]:
import random, shutil

In [ ]:
data_n_path = '/content/drive/MyDrive/Car_Images.zip (Unzipped Files)/normal'
data_ab_path = '/content/drive/MyDrive/Car_Images.zip (Unzipped Files)/abnormal'

In [ ]:
# 전체 이미지 갯수를 확인합니다.
len(os.listdir(data_n_path)) , len(os.listdir(data_ab_path))

(302, 303)

In [ ]:
# test 사이즈 : 전체 이미지의 20%
te_data_num = [round(len(os.listdir(data_n_path))*0.2), round(len(os.listdir(data_ab_path))*0.2)]
print(te_data_num)

# validation 사이즈 : test를 제외한 나머지 중에서 20%
val_data_num = [ round((len(os.listdir(data_n_path))-te_data_num[0])*0.2) , round((len(os.listdir(data_n_path))-te_data_num[1])*0.2) ]
print(val_data_num)

# train 사이즈
train_data_num = [len(os.listdir(data_n_path)) - te_data_num[0] - val_data_num[0],
                  len(os.listdir(data_ab_path))- te_data_num[1] - val_data_num[1]]
print(train_data_num)

[60, 61]
[48, 48]
[194, 194]


#### 2) test 셋 추출

In [ ]:
import numpy as np
import glob
import cv2

In [ ]:
normal_data = os.listdir(data_n_path)

In [ ]:
abnormal_data = os.listdir(data_ab_path)

In [ ]:
np.random.seed(2023)
random.shuffle(normal_data)
random.shuffle(abnormal_data )
test_n = normal_data[:te_data_num[0]]
test_ab = abnormal_data[:te_data_num[1]]

In [ ]:
# 추출 후 이미지 갯수 확인
print(len(test_n), len(test_ab))

60 61


In [ ]:
test_n_path = '/content/drive/MyDrive/datasets/Car_Images_test/normal'
test_ab_path = '/content/drive/MyDrive/datasets/Car_Images_test/abnormal'
val_n_path = '/content/drive/MyDrive/datasets/Car_Images_val/normal'
val_ab_path = '/content/drive/MyDrive/datasets/Car_Images_val/abnormal'
train_n_path = '/content/drive/MyDrive/datasets/Car_Images_train/normal'
train_ab_path = '/content/drive/MyDrive/datasets/Car_Images_train/abnormal'

In [ ]:
for i in test_n :
    src_path = os.path.join('/content/drive/MyDrive/Car_Images.zip (Unzipped Files)/normal', i)
    dst_path = os.path.join(test_n_path, i)
    shutil.copy(src_path, dst_path)

for i in test_ab :
    src_path = os.path.join('/content/drive/MyDrive/Car_Images.zip (Unzipped Files)/abnormal', i)
    dst_path = os.path.join(test_ab_path, i)
    shutil.copy(src_path, dst_path)

#### 3) validation 셋 추출

In [ ]:
val_n = normal_data[te_data_num[0]:val_data_num[0]+te_data_num[0]]
val_ab = abnormal_data[te_data_num[1]:val_data_num[1]+te_data_num[1]]

print(len(val_n), len(val_ab))


48 48


In [ ]:
for i in val_n :
    src_path = os.path.join('/content/drive/MyDrive/Car_Images.zip (Unzipped Files)/normal', i)
    dst_path = os.path.join(val_n_path, i)
    shutil.copy(src_path, dst_path)

for i in val_ab :
    src_path = os.path.join('/content/drive/MyDrive/Car_Images.zip (Unzipped Files)/abnormal', i)
    dst_path = os.path.join(val_ab_path, i)
    shutil.copy(src_path, dst_path)

- train

In [ ]:
train_n = normal_data[val_data_num[0]+te_data_num[0]:]
train_ab = abnormal_data[val_data_num[1]+te_data_num[1]:]

In [ ]:
print(len(train_n), len(train_ab))

194 194


In [ ]:
for i in train_n :
    src_path = os.path.join('/content/drive/MyDrive/Car_Images.zip (Unzipped Files)/normal', i)
    dst_path = os.path.join(train_n_path, i)
    shutil.copy(src_path, dst_path)

for i in train_ab :
    src_path = os.path.join('/content/drive/MyDrive/Car_Images.zip (Unzipped Files)/abnormal', i)
    dst_path = os.path.join(train_ab_path, i)
    shutil.copy(src_path, dst_path)

### (2) 데이터 복사 및 이동
- **세부요구사항**
    - 분할된 데이터를 복사 이동합니다.
        - 새로운 폴더에 저장하는 데이터로 "3.모델링I"에서 사용합니다.
        - 기존 폴더는 "4.모델링II > (1) Data Augmentation"에서 사용합니다.
    - Training set | Validation set | Test set의 데이터를 **새로운 폴더**에 복사하세요.
        - 새로운 폴더 명
            * copy_images/trainset
            * copy_images/validset
            * copy_images/testset
        - 새로운 폴더에는 normal, abnormal 파일 모두를 복사합니다. 
            * 파일을 구분하기 위해 abnormal 파일들은 파일명 앞에 접두사 'ab_'를 붙입시다.
        - os, shutil 모듈을 활용하세요.

#### 1) abnormal 파일 복사

In [ ]:
from distutils.dir_util import copy_tree

* 복사하기 : shutil.copytree()

In [ ]:
def rename1(data, paths):
    for i in data :
        old = os.path.join(paths, i)
        new = os.path.join(paths, 'ab_'+i)
        os.rename(old, new)

In [ ]:
rename1(val_ab, val_ab_path)
rename1(test_ab, test_ab_path)
rename1(train_ab, train_ab_path)

In [ ]:
!mkdir /content/drive/MyDrive/copy_images/; mkdir /content/drive/MyDrive/copy_images/testset/
!mkdir /content/drive/MyDrive/copy_images/; mkdir /content/drive/MyDrive/copy_images/trainset/
!mkdir /content/drive/MyDrive/copy_images/; mkdir /content/drive/MyDrive/copy_images/validset/

mkdir: cannot create directory ‘/content/drive/MyDrive/copy_images/’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/copy_images/’: File exists


In [ ]:
copy_tree("/content/drive/MyDrive/datasets/Car_Images_test/abnormal", "/content/drive/MyDrive/copy_images/testset")
copy_tree("/content/drive/MyDrive/datasets/Car_Images_train/abnormal", "/content/drive/MyDrive/copy_images/trainset")
copy_tree("/content/drive/MyDrive/datasets/Car_Images_val/abnormal", "/content/drive/MyDrive/copy_images/validset")

['/content/drive/MyDrive/copy_images/validset/ab_DALL¡¤E 2023-03-11 15.12.31 - dents of a car.png',
 '/content/drive/MyDrive/copy_images/validset/ab_DALL¡¤E 2023-03-11 15.00.43 - dents of a car.png',
 '/content/drive/MyDrive/copy_images/validset/ab_DALL¡¤E 2023-03-11 17.27.06 - slightly dented car.png',
 '/content/drive/MyDrive/copy_images/validset/ab_DALL¡¤E 2023-03-11 17.25.14 - slightly dented car.png',
 '/content/drive/MyDrive/copy_images/validset/ab_DALL¡¤E 2023-03-11 15.13.25 - dents of a car.png',
 '/content/drive/MyDrive/copy_images/validset/ab_DALL¡¤E 2023-03-11 15.06.35 - dents of a car.png',
 '/content/drive/MyDrive/copy_images/validset/ab_DALL¡¤E 2023-03-11 00.14.22 - a part of car without scratch.png',
 '/content/drive/MyDrive/copy_images/validset/ab_DALL¡¤E 2023-03-11 15.08.48 - dents of a car.png',
 '/content/drive/MyDrive/copy_images/validset/ab_DALL¡¤E 2023-03-10 22.04.39 - scratched car.png',
 '/content/drive/MyDrive/copy_images/validset/ab_DALL¡¤E 2023-03-11 01.31.38

* abnormal 이미지 이름의 접두어 "ab_" 붙이기 : os.rename

    - li = [test_ab, train_ab, val_ab]
    - paths = [test_ab_path, train_ab_path, val_ab_path]

#### 2) normal 파일 복사

In [ ]:
copy_tree("/content/drive/MyDrive/datasets/Car_Images_test/normal", "/content/drive/MyDrive/copy_images/testset")
copy_tree("/content/drive/MyDrive/datasets/Car_Images_train/normal", "/content/drive/MyDrive/copy_images/trainset")
copy_tree("/content/drive/MyDrive/datasets/Car_Images_val/normal", "/content/drive/MyDrive/copy_images/validset")

['/content/drive/MyDrive/copy_images/validset/DALL¡¤E 2023-03-11 01.36.44 - a part of a car.png',
 '/content/drive/MyDrive/copy_images/validset/DALL¡¤E 2023-03-11 14.26.19 - part of a car.png',
 '/content/drive/MyDrive/copy_images/validset/DALL¡¤E 2023-03-10 23.31.56 - photo of a part of car without blemish.png',
 '/content/drive/MyDrive/copy_images/validset/DALL¡¤E 2023-03-10 23.31.20 - photo of a part of car without blemish.png',
 '/content/drive/MyDrive/copy_images/validset/DALL¡¤E 2023-03-11 14.25.48 - part of a car.png',
 '/content/drive/MyDrive/copy_images/validset/DALL¡¤E 2023-03-10 23.35.37 - photo of a part of car without blemish.png',
 '/content/drive/MyDrive/copy_images/validset/DALL¡¤E 2023-03-11 14.37.35 - photo of part of a car.png',
 '/content/drive/MyDrive/copy_images/validset/DALL¡¤E 2023-03-11 00.57.21 - photo of a part of car.png',
 '/content/drive/MyDrive/copy_images/validset/DALL¡¤E 2023-03-10 22.23.54 - photo of a part of car.png',
 '/content/drive/MyDrive/copy_im

* 데이터 갯수 조회

In [ ]:
print(len(os.listdir('/content/drive/MyDrive/copy_images/testset')))
print(len(os.listdir('/content/drive/MyDrive/copy_images/trainset')))
print(len(os.listdir('/content/drive/MyDrive/copy_images/validset')))

121
388
96


## 3.모델링 I
* **세부요구사항**
    * 모델링을 위한 데이터 구조 만들기
        * x : 이미지를 array로 변환합니다.
        * y : 이미지 갯수만큼 normal - 0, abnormal - 1 로 array를 만듭니다.
    * 모델을 최소 3개 이상 만들고 성능을 비교합니다.
        * 모델 학습 과정에 알맞은 보조 지표를 사용하세요.
        * 전처리 과정에서 생성한 Validation set을 적절하게 사용하세요.
        * Early Stopping을 반드시 사용하세요.
            * 최적의 가중치를 모델에 적용하세요.

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.callbacks import ReduceLROnPlateau

### (1) X : image to array
- **세부요구사항**
    * 모델링을 위해서는 np.array 형태로 데이터셋을 만들어야 합니다.
    * Training set / Validation set / Test set의 X는 이미지 형태로 되어있습니다. 
    * 이미지 파일을 불러와 train, valid, test 각각 array 형태로 변환해 봅시다.
        * 각 폴더로 부터 이미지 목록을 만들고
        * 이미지 한장씩 적절한 크기로 로딩하여 (keras.utils.load_img)
            * 이미지가 너무 크면 학습시간이 많이 걸리고, 메모리 부족현상이 발생될 수 있습니다.
            * 이미지 크기를 280 * 280 * 3 이내의 크기를 설정하여 로딩하시오.
            * array로 변환 (keras.utils.img_to_array, np.expand_dims)
        * 데이터셋에 추가합니다.(데이터셋도 array)

#### 1) 이미지 목록 만들기
* train, validation, test 폴더로 부터 이미지 목록을 생성합니다.

In [4]:
# 이미지 목록 저장
img_train_list = os.listdir('/content/drive/MyDrive/copy_images/trainset')
img_valid_list = os.listdir('/content/drive/MyDrive/copy_images/validset')
img_test_list = os.listdir('/content/drive/MyDrive/copy_images/testset')

In [5]:
# 메모리, 처리시간을 위해서 이미지 크기 조정
img_size = 280 ## 사이즈 조정 가능

#### 2) 이미지들을 배열 데이터셋으로 만들기

In [6]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt

In [7]:
files = glob.glob('/content/drive/MyDrive/copy_images/trainset/*')

In [8]:
images = []

for path in files:
    img = image.load_img(path, target_size=(img_size,img_size) )
    img = image.img_to_array(img)
    images.append(img)
    
    # plt.imshow(image.load_img(path))
    # plt.show()
    x_train = np.array(images)

In [9]:
files = glob.glob('/content/drive/MyDrive/copy_images/testset/*')

In [10]:
images = []

for path in files:
    img = image.load_img(path, target_size=(img_size,img_size) )
    img = image.img_to_array(img)
    images.append(img)
    
    # plt.imshow(image.load_img(path))
    # plt.show()
    x_test = np.array(images)

In [11]:
files = glob.glob('/content/drive/MyDrive/copy_images/validset/*')

In [12]:
images = []

for path in files:
    img = image.load_img(path, target_size=(img_size,img_size) )
    img = image.img_to_array(img)
    images.append(img)
    
    # plt.imshow(image.load_img(path))
    # plt.show()
    x_val = np.array(images)

In [13]:
x_train.shape, x_val.shape, x_test.shape

((388, 280, 280, 3), (96, 280, 280, 3), (121, 280, 280, 3))

### (2) y : 클래스 만들기
- **세부요구사항**
    - Training set / Validation set / Test set의 y를 생성합니다.
        - 각각 normal, abnormal 데이터의 갯수를 다시 확인하고
        - normal을 0, abnormal을 1로 지정합니다.

In [14]:
# 데이터 갯수 확인
print( len(img_train_list) )
print( len([val for val in img_train_list if val.startswith('ab_')]) )
print('---')
print( len(img_valid_list) )
print( len([val for val in img_valid_list if val.startswith('ab_')]) )
print('---')
print( len(img_test_list) )
print( len([val for val in img_test_list if val.startswith('ab_')]) )

388
194
---
96
48
---
121
61


* y_train, y_valid, y_test 만들기
    * normal, abnormal 데이터의 갯수를 다시 확인하고 normal을 0, abnormal을 1로 지정합니다.

In [15]:
y_train = []
for name in img_train_list:
    if 'ab_' in str(name) :
        y_train.append(1)
    else :
        y_train.append(0)

y_train = np.array(y_train)

In [16]:
y_test= []
for name in img_test_list:
    if 'ab_' in str(name) :
        y_test.append(1)
    else :
        y_test.append(0)

y_test = np.array(y_test)

In [17]:
y_valid= []
for name in img_valid_list:
    if 'ab_' in str(name) :
        y_valid.append(1)
    else :
        y_valid.append(0)

y_val = np.array(y_valid)

In [18]:
print(y_train.shape, y_test.shape, y_val.shape)

(388,) (121,) (96,)


### (3) 모델1
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 validation_data로 validation set을 사용하시오.
    - 반드시 Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

min-max scaling

In [19]:
mean_n, std_n = x_train.mean(), x_train.std()
x_train = (x_train - mean_n) / std_n
x_test = (x_test - mean_n) / std_n
x_val = (x_val - mean_n) / std_n

In [20]:
x_train.mean(), x_train.std()

(5.1392874e-07, 1.0)

categorical

In [21]:
class_n = len(np.unique(y_train))

In [22]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, class_n)
y_test = to_categorical(y_test, class_n)
y_val = to_categorical(y_val, class_n)

#### 1) 구조 설계

In [23]:
from tensorflow.keras.backend import clear_session
from tensorflow.keras.layers import Dense, Input, Flatten, BatchNormalization, Dropout, Conv2D, AvgPool2D, MaxPool2D
from tensorflow.keras.models import Model, Sequential

#### 2) 학습
* EarlyStopping 설정하고 학습시키기

In [31]:
# 세션클리어
clear_session()

# 모델 layer 엮기
il = Input(shape=(280, 280, 3))

cl = Conv2D(filters = 6,
            kernel_size = (5,5),
            strides = (1,1),
            padding = 'same',
            activation = 'relu')(il)
pl = AvgPool2D(pool_size = (2,2),
               strides = (2,2))(cl)
bl = BatchNormalization()(pl)

cl = Conv2D(filters = 16,
            kernel_size = (5,5),
            strides = (1,1),
            padding =  'valid',
            activation = 'relu')(bl)
pl = AvgPool2D(pool_size = (2,2),
               strides = (2,2))(cl)
bl = BatchNormalization()(pl)

fl = Flatten()(bl)
hl = Dense(120, activation = 'relu')(fl)
bl = BatchNormalization()(hl)

hl = Dense(84, activation = 'relu')(bl)
bl = BatchNormalization()(hl)
dl = Dropout(0.2)(bl)

ol = Dense(class_n, activation = 'sigmoid')(dl)

# 모델 처음, 끝 지정
model = Model(il, ol)

# 모델 compile
model.compile(loss = keras.losses.binary_crossentropy,
              metrics = ['accuracy'],
              optimizer = 'rmsprop')

# 양념
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 280, 280, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 280, 280, 6)       456       
                                                                 
 average_pooling2d (AverageP  (None, 140, 140, 6)      0         
 ooling2D)                                                       
                                                                 
 batch_normalization (BatchN  (None, 140, 140, 6)      24        
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 136, 136, 16)      2416      
                                                                 
 average_pooling2d_1 (Averag  (None, 68, 68, 16)       0     

In [32]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

In [33]:
es = EarlyStopping(monitor = 'val_loss',
                   patience = 10,
                   min_delta = 0, 
                   verbose = 1)

In [34]:
mcp = ModelCheckpoint(filepath = '/content/drive/MyDrive/Lenet.h5', # 모델 저장 경로
						monitor = 'val_loss' ,               # 모델 저장의 관심 대상
						verbose = 1,                         # 어느 시점에서 저장되는지 알려줌
						save_best_only = True,               # 최고 성능 모델만 저장
						save_weights_only = False)           # 가중치만 저장할 것인지, 모델 구조를 저장할 것인지

In [35]:
# 데이터를 넣어서 학습시키자!
history = model.fit(x_train, y_train,
                    epochs = 10000, verbose = 1,
                    callbacks = [es, mcp], validation_data = [x_val, y_val])

Epoch 1/10000
12/13 [==========================>...] - ETA: 0s - loss: 0.5387 - accuracy: 0.7474
Epoch 1: val_loss improved from inf to 1.06939, saving model to /content/drive/MyDrive/Lenet.h5
13/13 [==============================] - 5s 192ms/step - loss: 0.5382 - accuracy: 0.7474 - val_loss: 1.0694 - val_accuracy: 0.5521
Epoch 2/10000
12/13 [==========================>...] - ETA: 0s - loss: 0.3304 - accuracy: 0.8724
Epoch 2: val_loss improved from 1.06939 to 0.77307, saving model to /content/drive/MyDrive/Lenet.h5
13/13 [==============================] - 1s 76ms/step - loss: 0.3330 - accuracy: 0.8711 - val_loss: 0.7731 - val_accuracy: 0.5729
Epoch 3/10000
11/13 [========================>.....] - ETA: 0s - loss: 0.2290 - accuracy: 0.9290
Epoch 3: val_loss improved from 0.77307 to 0.57443, saving model to /content/drive/MyDrive/Lenet.h5
13/13 [==============================] - 1s 76ms/step - loss: 0.2432 - accuracy: 0.9201 - val_loss: 0.5744 - val_accuracy: 0.7396
Epoch 4/10000
13/13 [=

#### 3) test set으로 예측하고 평가하기
* 평가는 confusion_matrix, classification_report 활용

In [36]:
from sklearn.metrics import confusion_matrix, classification_report
y_pred = model.predict(x_test)
y_pred.shape
single_y_pred = y_pred.argmax(axis=1)
single_y_pred.shape
print(y_test.shape)
single_test_y = y_test.argmax(axis=1)
print(single_test_y.shape)
print(confusion_matrix(single_test_y, single_y_pred))
print(classification_report(single_test_y, single_y_pred))

4/4 [==============================] - 0s 16ms/step
(121, 2)
(121,)
[[50 10]
 [ 9 52]]
              precision    recall  f1-score   support

           0       0.85      0.83      0.84        60
           1       0.84      0.85      0.85        61

    accuracy                           0.84       121
   macro avg       0.84      0.84      0.84       121
weighted avg       0.84      0.84      0.84       121



### (4) 모델2
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 validation_data로 validation set을 사용하시오.
    - 반드시 Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

#### 1) 구조 설계

In [184]:
# 세션클리어
clear_session()

# 모델 layer 엮기
il = Input(shape=(280, 280, 3))

cl = Conv2D(filters = 96,
            kernel_size = (10,10),
            strides = (4,4),
            padding = 'valid',
            activation = 'relu')(il)
pl = MaxPool2D(pool_size = (3,3),
               strides = (2,2))(cl)
bl = BatchNormalization()(pl)

cl = Conv2D(filters = 256,
            kernel_size = (5,5),
            strides = (1,1),
            padding =  'same',
            activation = 'relu')(bl)
pl = MaxPool2D(pool_size = (2,2),
               strides = (1,1))(cl)
bl = BatchNormalization()(pl)

cl = Conv2D(filters = 384,
            kernel_size = (2,2),
            strides = (1,1),
            padding =  'same',
            activation = 'relu')(bl)

cl = Conv2D(filters = 256,
            kernel_size = (2,2),
            strides = (1,1),
            padding =  'same',
            activation = 'relu')(cl)

pl = MaxPool2D(pool_size = (2,2),
               strides = (2,2))(cl)
bl = BatchNormalization()(pl)

fl = Flatten()(bl)
hl = Dense(128, activation = 'relu')(fl)
bl = BatchNormalization()(hl)

hl = Dense(32, activation = 'relu')(bl)
bl = BatchNormalization()(hl)
dl = Dropout(0.3)(bl)

ol = Dense(class_n, activation = 'sigmoid')(dl)

# 모델 처음, 끝 지정
model = Model(il, ol)

# 모델 compile
model.compile(loss = keras.losses.binary_crossentropy,
              metrics = ['accuracy'],
              optimizer = 'rmsprop')

# 양념
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 280, 280, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 68, 68, 96)        28896     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 33, 33, 96)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 33, 33, 96)       384       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 33, 33, 256)       614656    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 256)      0     

#### 2) 학습
* EarlyStopping 설정하고 학습시키기

In [185]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

In [186]:
es = EarlyStopping(monitor = 'val_loss',
                   patience = 10,
                   min_delta = 0, 
                   verbose = 1)

In [187]:
# mcp = ModelCheckpoint(filepath = '/content/drive/MyDrive/Alexnet2.h5', # 모델 저장 경로
# 						monitor = 'val_loss' ,               # 모델 저장의 관심 대상
# 						verbose = 1,                         # 어느 시점에서 저장되는지 알려줌
# 						save_best_only = True,               # 최고 성능 모델만 저장
# 						save_weights_only = True)           # 가중치만 저장할 것인지, 모델 구조를 저장할 것인지

In [188]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.01, min_delta=0,
                              patience=10, min_lr=0)

In [189]:
# 데이터를 넣어서 학습시키자!
history = model.fit(x_train, y_train,
                    epochs = 10000, verbose = 1,
                    callbacks = [es, reduce_lr], validation_data = [x_val, y_val])

Epoch 1/10000
13/13 [==============================] - 4s 111ms/step - loss: 0.5432 - accuracy: 0.7784 - val_loss: 10.3608 - val_accuracy: 0.5312 - lr: 0.0010
Epoch 2/10000
13/13 [==============================] - 1s 83ms/step - loss: 0.4630 - accuracy: 0.8119 - val_loss: 3.5174 - val_accuracy: 0.6771 - lr: 0.0010
Epoch 3/10000
13/13 [==============================] - 1s 85ms/step - loss: 0.3500 - accuracy: 0.8789 - val_loss: 3.0588 - val_accuracy: 0.6771 - lr: 0.0010
Epoch 4/10000
13/13 [==============================] - 1s 85ms/step - loss: 0.3577 - accuracy: 0.8402 - val_loss: 6.9186 - val_accuracy: 0.5312 - lr: 0.0010
Epoch 5/10000
13/13 [==============================] - 1s 82ms/step - loss: 0.4170 - accuracy: 0.8299 - val_loss: 3.3018 - val_accuracy: 0.5625 - lr: 0.0010
Epoch 6/10000
13/13 [==============================] - 1s 87ms/step - loss: 0.3164 - accuracy: 0.8918 - val_loss: 3.2768 - val_accuracy: 0.5625 - lr: 0.0010
Epoch 7/10000
13/13 [==============================] - 1

#### 3) test set으로 예측하고 평가하기
* 평가는 confusion_matrix, classification_report 활용

In [190]:
from sklearn.metrics import confusion_matrix, classification_report
y_pred = model.predict(x_test)
y_pred.shape
single_y_pred = y_pred.argmax(axis=1)
single_y_pred.shape
print(y_test.shape)
single_test_y = y_test.argmax(axis=1)
print(single_test_y.shape)
print(confusion_matrix(single_test_y, single_y_pred))
print(classification_report(single_test_y, single_y_pred))

4/4 [==============================] - 0s 25ms/step
(121, 2)
(121,)
[[59  1]
 [21 40]]
              precision    recall  f1-score   support

           0       0.74      0.98      0.84        60
           1       0.98      0.66      0.78        61

    accuracy                           0.82       121
   macro avg       0.86      0.82      0.81       121
weighted avg       0.86      0.82      0.81       121



### (5) 모델3
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 validation_data로 validation set을 사용하시오.
    - 반드시 Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

#### 1) 구조 설계

In [166]:
# 세션 클리어
clear_session()

# 모델 레이어 엮기
il = Input(shape=(280, 280, 3))

cl = Conv2D(filters = 64,
            kernel_size = (3,3),
            padding = 'same',
            activation = 'relu')(il)

cl = Conv2D(filters = 64,
            kernel_size = (3,3),
            padding = 'same',
            activation = 'relu')(cl)
pl = MaxPool2D(pool_size=(2,2))(cl)
bl = BatchNormalization()(pl)

cl = Conv2D(filters = 128,
            kernel_size = (3,3),
            padding = 'same',
            activation = 'relu')(bl)
cl = Conv2D(filters = 128,
            kernel_size = (3,3),
            padding = 'same',
            activation = 'relu')(cl)
pl = MaxPool2D(pool_size=(2,2))(cl)
bl = BatchNormalization()(pl)

fl = Flatten()(bl)
hl = Dense(64)(fl)
bl = BatchNormalization()(hl)
dl = Dropout(0.3)(bl)

ol = Dense(2, activation = 'sigmoid')(dl)

# 모델 처음, 끝 지정
model = Model(il, ol )

# 모델 compile
model.compile(loss = keras.losses.binary_crossentropy,
              metrics = ['accuracy'],
              optimizer = 'rmsprop')

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 280, 280, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 280, 280, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 280, 280, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 140, 140, 64)     0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 140, 140, 64)     256       
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 140, 140, 128)     73856 

#### 2) 학습
* EarlyStopping 설정하고 학습시키기

In [167]:
es = EarlyStopping(monitor = 'val_loss',
                   patience = 10,
                   min_delta = 0, 
                   verbose = 1)

In [168]:
# mcp = ModelCheckpoint(filepath = '/content/drive/MyDrive/Vgget.h5', # 모델 저장 경로
# 						monitor = 'val_loss' ,               # 모델 저장의 관심 대상
# 						verbose = 1,                         # 어느 시점에서 저장되는지 알려줌
# 						save_best_only = True,               # 최고 성능 모델만 저장
# 						save_weights_only = False)           # 가중치만 저장할 것인지, 모델 구조를 저장할 것인지

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.01, min_delta=0,
                              patience=10, min_lr=0)

In [169]:
# 데이터를 넣어서 학습시키자!
history = model.fit(x_train, y_train,
                    epochs = 10000, verbose = 1,
                    callbacks = [es, reduce_lr], validation_data = [x_val, y_val])

Epoch 1/10000
13/13 [==============================] - 7s 367ms/step - loss: 0.8566 - accuracy: 0.7062 - val_loss: 3.8625 - val_accuracy: 0.5000
Epoch 2/10000
13/13 [==============================] - 4s 332ms/step - loss: 0.4896 - accuracy: 0.8144 - val_loss: 1.6128 - val_accuracy: 0.6042
Epoch 3/10000
13/13 [==============================] - 4s 330ms/step - loss: 0.4682 - accuracy: 0.8402 - val_loss: 0.7153 - val_accuracy: 0.8438
Epoch 4/10000
13/13 [==============================] - 4s 333ms/step - loss: 0.3556 - accuracy: 0.8711 - val_loss: 1.1261 - val_accuracy: 0.7917
Epoch 5/10000
13/13 [==============================] - 4s 332ms/step - loss: 0.2804 - accuracy: 0.9046 - val_loss: 0.7210 - val_accuracy: 0.8333
Epoch 6/10000
13/13 [==============================] - 4s 334ms/step - loss: 0.2720 - accuracy: 0.8892 - val_loss: 1.0658 - val_accuracy: 0.7292
Epoch 7/10000
13/13 [==============================] - 4s 338ms/step - loss: 0.2244 - accuracy: 0.9330 - val_loss: 0.5266 - val_ac

#### 3) test set으로 예측하고 평가하기
* 평가는 confusion_matrix, classification_report 활용

In [170]:
from sklearn.metrics import confusion_matrix, classification_report
y_pred = model.predict(x_test)
y_pred.shape
single_y_pred = y_pred.argmax(axis=1)
single_y_pred.shape
print(y_test.shape)
single_test_y = y_test.argmax(axis=1)
print(single_test_y.shape)
print(confusion_matrix(single_test_y, single_y_pred))
print(classification_report(single_test_y, single_y_pred))

4/4 [==============================] - 1s 112ms/step
(121, 2)
(121,)
[[50 10]
 [10 51]]
              precision    recall  f1-score   support

           0       0.83      0.83      0.83        60
           1       0.84      0.84      0.84        61

    accuracy                           0.83       121
   macro avg       0.83      0.83      0.83       121
weighted avg       0.83      0.83      0.83       121



In [171]:
model = Sequential()
model.add(Conv2D(32 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = (280,280,3)))
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(128 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(256 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Flatten())
model.add(Dense(units = 128 , activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(units = 2 , activation = 'sigmoid'))
model.compile(optimizer = "rmsprop" , loss = 'binary_crossentropy' , metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 280, 280, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 140, 140, 32)     0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 140, 140, 64)      18496     
                                                                 
 batch_normalization_3 (Batc  (None, 140, 140, 64)     256       
 hNormalization)                                                 
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 70, 70, 64)       0         
 2D)                                                             
                                                        

In [174]:
es = EarlyStopping(monitor = 'val_loss',
                   patience = 10,
                   min_delta = 0, 
                   verbose = 1)

In [175]:
# mcp = ModelCheckpoint(filepath = '/content/drive/MyDrive/kaggle.h5', # 모델 저장 경로
# 						monitor = 'val_loss' ,               # 모델 저장의 관심 대상
# 						verbose = 1,                         # 어느 시점에서 저장되는지 알려줌
# 						save_best_only = True,               # 최고 성능 모델만 저장
# 						save_weights_only = False)           # 가중치만 저장할 것인지, 모델 구조를 저장할 것인지

In [176]:
# 데이터를 넣어서 학습시키자!
history = model.fit(x_train, y_train,
                    epochs = 10000, verbose = 1,
                    callbacks = [es, reduce_lr], validation_data = [x_val, y_val])

Epoch 1/10000
13/13 [==============================] - 5s 122ms/step - loss: 4.0961 - accuracy: 0.6933 - val_loss: 0.5903 - val_accuracy: 0.7812
Epoch 2/10000
13/13 [==============================] - 1s 93ms/step - loss: 0.5008 - accuracy: 0.8376 - val_loss: 0.5443 - val_accuracy: 0.8125
Epoch 3/10000
13/13 [==============================] - 1s 93ms/step - loss: 0.4950 - accuracy: 0.8351 - val_loss: 0.6535 - val_accuracy: 0.8021
Epoch 4/10000
13/13 [==============================] - 1s 93ms/step - loss: 0.3823 - accuracy: 0.8814 - val_loss: 0.9523 - val_accuracy: 0.7292
Epoch 5/10000
13/13 [==============================] - 1s 94ms/step - loss: 0.4369 - accuracy: 0.8479 - val_loss: 0.8363 - val_accuracy: 0.8229
Epoch 6/10000
13/13 [==============================] - 1s 93ms/step - loss: 0.2480 - accuracy: 0.9046 - val_loss: 1.4302 - val_accuracy: 0.7708
Epoch 7/10000
13/13 [==============================] - 1s 91ms/step - loss: 0.2234 - accuracy: 0.9227 - val_loss: 1.2737 - val_accuracy

In [177]:
from sklearn.metrics import confusion_matrix, classification_report
y_pred = model.predict(x_test)
y_pred.shape
single_y_pred = y_pred.argmax(axis=1)
single_y_pred.shape
print(y_test.shape)
single_test_y = y_test.argmax(axis=1)
print(single_test_y.shape)
print(confusion_matrix(single_test_y, single_y_pred))
print(classification_report(single_test_y, single_y_pred))

4/4 [==============================] - 0s 26ms/step
(121, 2)
(121,)
[[35 25]
 [ 7 54]]
              precision    recall  f1-score   support

           0       0.83      0.58      0.69        60
           1       0.68      0.89      0.77        61

    accuracy                           0.74       121
   macro avg       0.76      0.73      0.73       121
weighted avg       0.76      0.74      0.73       121



In [24]:
model_path='/content/drive/MyDrive/first_model.h5'
# 모델 파일 불러오기
model = tf.keras.models.load_model(model_path)

from sklearn.metrics import confusion_matrix, classification_report
y_pred = model.predict(x_test)
y_pred.shape
single_y_pred = y_pred.argmax(axis=1)
single_y_pred.shape
print(y_test.shape)
single_test_y = y_test.argmax(axis=1)
print(single_test_y.shape)
print(confusion_matrix(single_test_y, single_y_pred))
print(classification_report(single_test_y, single_y_pred))

4/4 [==============================] - 3s 207ms/step
(121, 2)
(121,)
[[60  0]
 [61  0]]
              precision    recall  f1-score   support

           0       0.50      1.00      0.66        60
           1       0.00      0.00      0.00        61

    accuracy                           0.50       121
   macro avg       0.25      0.50      0.33       121
weighted avg       0.25      0.50      0.33       121



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### (1) Data Augmentation
- **세부요구사항**
    * 모델 학습에 이용할 이미지 데이터를 증강시키세요.
    * Keras의 ImageDataGenerator를 이용
        - [ImageDataGenerator document](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator)

    * image generator를 이용하여 학습
        * 모델 구조는 이미 생성한 1,2,3 중 하나를 선택하여 학습


In [24]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input

In [25]:
img_size = 280 ## 사이즈 조정 가능

train_path = '/content/drive/MyDrive/datasets/Car_Images_train/'
valid_path = '/content/drive/MyDrive/datasets/Car_Images_val/'
test_path = '/content/drive/MyDrive/datasets/Car_Images_test/'

#### 1) ImageGenerator 생성
* ImageDataGenerator 함수 사용
    * 주요 옵션
        * rotation_range: 무작위 회전을 적용할 각도 범위
        * zoom_range: 무작위 줌을 적용할 범위 [1-zoom_range, 1+zoom_range]
        * horizontal_flip: 무작위 좌우반전을 적용할지 여부
        * vertical_flip: 무작위 상하반전을 적용할지 여부
        * rescale: 텐서의 모든 값을 rescale 값으로 나누어줌 (이 경우에는 255로 나누어서 0~1사이의 값으로 변경)

In [28]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range = 10,
                                   zoom_range = [0, 0.5],
                                   horizontal_flip=False,
                                   vertical_flip = False,
                                   width_shift_range = 0.1,
                                   height_shift_range = 0.1,
                                   fill_mode ='wrap', 
                                   rescale=1./255,)
valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range = 10,
                                   zoom_range = [0, 0.5],
                                   horizontal_flip=False,
                                   vertical_flip = False,
                                   width_shift_range = 0.1,
                                   height_shift_range = 0.1,
                                   fill_mode ='wrap', 
                                   rescale=1./255,)


#### 2) 경로로 부터 이미지 불러 올 준비
* .flow_from_directory 이용
    * 디렉토리에서 이미지를 가져와서 데이터 증강을 적용하고 batch 단위로 제공하는 generator를 생성합니다.
    * 이미지를 불러올 때 target_size로 크기를 맞추고, 
    * class_mode로 이진 분류(binary)를 수행하도록 지정합니다.


In [29]:
train_flow_gen = train_datagen.flow_from_directory(directory = train_path,
                                                   target_size = (280,280),
                                                   classes = ['normal', 'abnormal'],
                                                   class_mode = 'binary',
                                                   shuffle = True)
val_flow_gen = valid_datagen.flow_from_directory(directory= valid_path,
                                                 target_size = (280, 280),
                                                 classes = ['normal', 'abnormal'],
                                                 class_mode = 'binary',
                                                 shuffle = True)

Found 388 images belonging to 2 classes.
Found 96 images belonging to 2 classes.


#### 3) 학습
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 train_generator 이용. 
    - validation_data = valid_generator 지정
    - Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

* 구조 설계

* 학습
    * EarlyStopping 설정하기
    * 학습 데이터에 train_generator, validation_data=valid_generator 사용

In [30]:
# 세션클리어
clear_session()

# 모델 layer 엮기
il = Input(shape=(280, 280, 3))

cl = Conv2D(filters = 6,
            kernel_size = (5,5),
            strides = (1,1),
            padding = 'same',
            activation = 'relu')(il)
pl = AvgPool2D(pool_size = (2,2),
               strides = (2,2))(cl)
bl = BatchNormalization()(pl)

cl = Conv2D(filters = 16,
            kernel_size = (5,5),
            strides = (1,1),
            padding =  'valid',
            activation = 'relu')(bl)
pl = AvgPool2D(pool_size = (2,2),
               strides = (2,2))(cl)
bl = BatchNormalization()(pl)

fl = Flatten()(bl)
hl = Dense(120, activation = 'relu')(fl)
bl = BatchNormalization()(hl)

hl = Dense(84, activation = 'relu')(bl)
bl = BatchNormalization()(hl)
dl = Dropout(0.2)(bl)

ol = Dense(1, activation = 'sigmoid')(dl)

# 모델 처음, 끝 지정
model = Model(il, ol)

# 모델 compile
# optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(loss = keras.losses.binary_crossentropy,
              metrics = ['accuracy'],
              optimizer = 'rmsprop')

# 양념
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 280, 280, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 280, 280, 6)       456       
                                                                 
 average_pooling2d (AverageP  (None, 140, 140, 6)      0         
 ooling2D)                                                       
                                                                 
 batch_normalization (BatchN  (None, 140, 140, 6)      24        
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 136, 136, 16)      2416      
                                                                 
 average_pooling2d_1 (Averag  (None, 68, 68, 16)       0     

In [31]:
from tensorflow.keras.callbacks import EarlyStopping

In [32]:
es = EarlyStopping(monitor = 'val_loss',
                   patience = 10,
                   min_delta = 0, 
                   verbose = 1)

In [33]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=10, min_lr=0)

In [ ]:
# 데이터를 넣어서 학습시키자!
history = model.fit(train_flow_gen,
                    epochs = 10000, verbose = 1,batch_size = 100,
                    callbacks = [es, reduce_lr], validation_data = [val_flow_gen])

Epoch 1/10000
13/13 [==============================] - 33s 2s/step - loss: 0.6519 - accuracy: 0.6856 - val_loss: 0.6776 - val_accuracy: 0.5729 - lr: 0.0010
Epoch 2/10000
13/13 [==============================] - 24s 2s/step - loss: 0.6236 - accuracy: 0.6933 - val_loss: 0.6731 - val_accuracy: 0.6146 - lr: 0.0010
Epoch 3/10000
13/13 [==============================] - 24s 2s/step - loss: 0.6124 - accuracy: 0.6624 - val_loss: 0.6371 - val_accuracy: 0.6250 - lr: 0.0010
Epoch 4/10000
13/13 [==============================] - 25s 2s/step - loss: 0.6196 - accuracy: 0.7165 - val_loss: 0.6407 - val_accuracy: 0.6667 - lr: 0.0010
Epoch 5/10000
13/13 [==============================] - 24s 2s/step - loss: 0.5298 - accuracy: 0.7474 - val_loss: 0.6365 - val_accuracy: 0.6979 - lr: 0.0010
Epoch 6/10000
13/13 [==============================] - 24s 2s/step - loss: 0.5260 - accuracy: 0.7423 - val_loss: 0.6743 - val_accuracy: 0.5729 - lr: 0.0010
Epoch 7/10000
13/13 [==============================] - 24s 2s/st

#### 4) 성능 평가
* 평가는 confusion_matrix, classification_report 활용

In [105]:
from sklearn.metrics import confusion_matrix, classification_report
y_pred = model.predict(x_test)
y_pred.shape
single_y_pred = y_pred.argmax(axis=1)
single_y_pred.shape
print(y_test.shape)
single_test_y = y_test.argmax(axis=1)
print(single_test_y.shape)
print(confusion_matrix(single_test_y, single_y_pred))
print(classification_report(single_test_y, single_y_pred))

4/4 [==============================] - 0s 22ms/step
(121, 2)
(121,)
[[60  0]
 [61  0]]
              precision    recall  f1-score   support

           0       0.50      1.00      0.66        60
           1       0.00      0.00      0.00        61

    accuracy                           0.50       121
   macro avg       0.25      0.50      0.33       121
weighted avg       0.25      0.50      0.33       121



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### (2) Transfer Learning
- **세부요구사항**
    * VGG16 모델은 1000개의 클래스를 분류하는 데 사용된 ImageNet 데이터셋을 기반으로 사전 학습된 가중치를 가지고 있습니다. 
        * 따라서 이 모델은 이미지 분류 문제에 대한 높은 성능을 보입니다.
        * 이 모델은 보통 전이학습(transfer learning)에서 기본적으로 사용되며, 특히 대규모 데이터셋이 없을 때는 기본 모델로 사용되어 fine-tuning을 수행합니다.
    * VGG16 함수로 부터 base_model 저장


In [ ]:
from tensorflow.keras.applications import VGG16

#### 1) VGG16 불러와서 저장하기
* include_top=False로 설정하여 분류기를 제외하고 미리 학습된 가중치 imagenet을 로드합니다.
* .trainable을 True로 설정하여 모델의 모든 레이어들이 fine-tuning에 대해 업데이트되도록 합니다.


In [ ]:
base_model = VGG16(                 )




#### 2) VGG16과 연결한 구조 설계
* VGG16을 불러와서 Flatten, Dense 등으로 레이어 연결하기

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model

# VGG16 모델 불러오기
vgg_model = VGG16(weights='imagenet',
                  include_top=False, 
                  input_shape=(280, 280, 3))

# 기존 모델과 VGG16 연결하기
last_layer = model.get_layer('dense_2').output
x = Flatten()(vgg_model.output)
x = Dense(512, activation='relu')(x)
x = Dense(2, activation='sigmoid')(x)
new_model = Model(inputs=vgg_model.input, outputs=x)

new_model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# VGG16 모델의 가중치 동결하기
for layer in vgg_model.layers:
    layer.trainable = False

history = new_model.fit(x_train, y_train, epochs = 10000, verbose = 1, callbacks = [es], validation_data =(x_val,y_val))

#### 3) 학습
- **세부요구사항**
    - 모델 학습 과정에 알맞은 보조 지표를 사용하세요.
    - 데이터
        * Image Generator를 연결하거나
        * 기존 train, validation 셋을 이용해도 됩니다.
        - Early Stopping을 반드시 사용하세요.
        - 최적의 가중치를 모델에 적용하세요.

#### 4) 성능 평가